In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from DataLoader import MyDataloader
from Engine import Engine
from Architecture import MainModel, PreModel

In [17]:
from pathlib import Path
import pandas as pd
import torch

In [37]:
path = Path(r"/mnt/Data1/Python_Projects/Pure-Python/P5/01-Thesis/data/Shiraze94_en")
data = [pd.read_csv(path/"activity_chain.csv"), pd.read_csv(path/"OD_matrix_area.csv"), pd.read_csv(path/"trips.csv")]
seed = 42

TRAIN_PERCENT, VAL_PERCENT, TEST_PERCENT = 0.7, 0.15, 0.15
BATCH_SIZE = 512
dataloader = MyDataloader(data_list=data, 
                          train_percent=TRAIN_PERCENT,
                          val_percent=VAL_PERCENT,
                          test_percent=TEST_PERCENT,
                          batch_size=BATCH_SIZE,
                          seed=seed)

train_data, val_data, test_data = dataloader.fit()

In [38]:
for batch, (x1, x2, x, y) in enumerate(train_data):
    print(x1.shape)
    print(x2.shape)
    print(x.shape)
    print(y.shape)
    break

torch.Size([512, 10, 5])
torch.Size([512, 15])
torch.Size([512, 3])
torch.Size([512])


In [43]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

x1_dim = 5
x2_dim = 15
x_dim = 3
EPOCHS = 1
LEARNING_RATE = 0.001

pre_model = PreModel(x1_dim=x1_dim,
                     x2_dim=x2_dim,
                     d_model=64,
                     nhead=4,
                     num_layers=2,
                     mlp_hidden=64,
                     out_dim=32)

main_model = MainModel(z_dim=32,
                       x_dim=x_dim,
                       hidden1=32,
                       hidden2=32)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer_pre = torch.optim.Adam(params=pre_model.parameters(), lr=LEARNING_RATE)
optimizer_main = torch.optim.Adam(params=main_model.parameters(), lr=LEARNING_RATE)


engine = Engine(model=main_model,
                pre_model=pre_model,
                train_dataloader=train_data,
                val_dataloader=val_data,
                test_dataloader=test_data,
                random_seed=42)
model0_results = engine.train(loss_fn=loss_fn,
                              optimizer_pre=optimizer_pre,
                              optimizer_main=optimizer_main,
                              epochs_num=EPOCHS,
                              resolution=1,
                              device="cuda")

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[9.9569e+04, 6.9000e+02, 3.0000e+00, 1.0510e+04, 0.0000e+00],
        [9.9570e+04, 1.4100e+03, 1.0000e+01, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0510e+04, 0.0000e+00]],
       device='cuda:0')
tensor([  0.,  54.,   5.,   0.,   1.,   4., 248.,   4.,   1.,   0.,   0.,   1.,
          0.,   0.,   0.], device='cuda:0')
tensor([  0.9943,  -2.5724,   2.3995,   3.9012,   0.7128,  -4.9943,  -7.2167,
          3.7615,   8.1732,   2.0093,   0.7934,  -1.5773,   2.8083,  -0.7

RuntimeError: No active exception to reraise